<a href="https://colab.research.google.com/github/lutakrystal305/DL_basic/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nttuan8/DL_Tutorial

Cloning into 'DL_Tutorial'...
remote: Enumerating objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 60.69 MiB | 23.47 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
!unzip /content/DL_Tutorial/L9/dataset.zip
%cd /content/

In [4]:
!pwd

/content


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense, Flatten, Dropout
import numpy as np
import random
import os

In [21]:
image_path = list(paths.list_images('dataset/'))
random.shuffle(image_path)
labels = [p.split(os.path.sep)[-2] for p in image_path]

le = LabelEncoder()
labels = le.fit_transform(labels)

#one hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [23]:
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)

    list_image.append(image)

list_image = np.vstack(list_image) 
print(list_image.shape)

(1360, 224, 224, 3)


In [27]:
#load model vgg16 reject fcl
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add layer

#Lấy output của convNet trong VGG16
fcHead = baseModel.output

#Flatten truoc khi dung FCs
fcHead = Flatten(name='flatten')(fcHead)

#Thêm Fully Connected Layer
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)
fcHead = Dense(17, activation='softmax')(fcHead)

print(baseModel.input)
#Xây dựng model bằng cách nối baseModel và fcHead
model = Model(inputs=baseModel.input, outputs = fcHead)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_9'), name='input_9', description="created by layer 'input_9'")


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [31]:
#data augmentation
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2,
                               zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
                               )
aug_test = ImageDataGenerator(rescale=1./255)


In [33]:
#Đóng băng các layer của VGG16
for layer in baseModel.layers:
    layer.trainable = False

opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
#git_generator vì dùng aug sẽ sinh ra thêm ảnh
H = model.fit_generator(aug_train.flow(X_train, Y_train, batch_size=32),
                        steps_per_epoch = len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, Y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs = numOfEpoch
                        )


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
34/34 [==============================] - 53s 321ms/step - loss: 6.7390 - accuracy: 0.1067 - val_loss: 2.2202 - val_accuracy: 0.2852
Epoch 2/25
34/34 [==============================] - 10s 304ms/step - loss: 2.5018 - accuracy: 0.2242 - val_loss: 1.7843 - val_accuracy: 0.4453
Epoch 3/25
34/34 [==============================] - 10s 306ms/step - loss: 2.0692 - accuracy: 0.3620 - val_loss: 1.2243 - val_accuracy: 0.6367
Epoch 4/25
34/34 [==============================] - 10s 306ms/step - loss: 1.7355 - accuracy: 0.4589 - val_loss: 0.9954 - val_accuracy: 0.7070
Epoch 5/25
34/34 [==============================] - 10s 307ms/step - loss: 1.5591 - accuracy: 0.4948 - val_loss: 0.8933 - val_accuracy: 0.7344
Epoch 6/25
34/34 [==============================] - 10s 306ms/step - loss: 1.3769 - accuracy: 0.5576 - val_loss: 0.9400 - val_accuracy: 0.6602
Epoch 7/25
34/34 [==============================] - 10s 307ms/step - loss: 1.2755 - accuracy: 0.5803 - val_loss: 0.7007 - val_accuracy: 0.8008

In [35]:
#phá băng các layer của VGG
for layer in baseModel.layers:
    layer.trainable = True

numOfEpoch = 35
opt = SGD(0.001)
H = model.fit_generator(aug_train.flow(X_train, Y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, Y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

34/34 [==============================] - 10s 305ms/step - loss: 0.4192 - accuracy: 0.8695 - val_loss: 0.4353 - val_accuracy: 0.8867
Epoch 28/35
34/34 [==============================] - 10s 303ms/step - loss: 0.3954 - accuracy: 0.8686 - val_loss: 0.5051 - val_accuracy: 0.8945
Epoch 29/35
34/34 [==============================] - 10s 304ms/step - loss: 0.3925 - accuracy: 0.8713 - val_loss: 0.4901 - val_accuracy: 0.8789
Epoch 30/35
34/34 [==============================] - 10s 304ms/step - loss: 0.3950 - accuracy: 0.8741 - val_loss: 0.4853 - val_accuracy: 0.8594
Epoch 31/35
34/34 [==============================] - 10s 306ms/step - loss: 0.4147 - accuracy: 0.8732 - val_loss: 0.5052 - val_accuracy: 0.8945
Epoch 32/35
34/34 [==============================] - 10s 306ms/step - loss: 0.4270 - accuracy: 0.8768 - val_loss: 0.4901 - val_accuracy: 0.8945
Epoch 33/35
34/34 [==============================] - 10s 304ms/step - loss: 0.4103 - accuracy: 0.8676 - val_loss: 0.5786 - val_accuracy: 0.8633
Epoc